# Import Agribalyse

In [1]:
import numpy as np
import pandas as pd
import brightway2 as bw2

In [2]:
# some functions
def find_location_code(row, tb_loc):
    val = row['F_LOCATION'] 
    querry = tb_loc.query("ID == @val")['CODE']
    if len(querry)>0:
        loc = querry.values[0]
    else:
        loc = ''
    return loc


In [3]:
# Step 1) Load the CSV data into memory
folder = 'agribalyse/'
files = ['TBL_EXCHANGES', 'TBL_PROCESSES', 'TBL_FLOWS', 'TBL_LOCATIONS', 'TBL_CATEGORIES']

tbl_processes = pd.read_csv(folder+'TBL_PROCESSES.csv', sep=',')
tbl_flows = pd.read_csv(folder+'TBL_FLOWS.csv', sep=',')
tbl_exchanges = pd.read_csv(folder+'TBL_EXCHANGES.csv', sep=',')
tbl_categories = pd.read_csv(folder+'TBL_CATEGORIES.csv')
tbl_locations = pd.read_csv(folder+'TBL_LOCATIONS.csv')

# tbl exchanges contains 75% of zeros !
# cf. https://github.com/lou-dupont/Agribalyse/commit/eea57713550a883662daaa03a531565223c43306 
tbl_exchanges = tbl_exchanges[tbl_exchanges['RESULTING_AMOUNT_VALUE'] != 0]

C:\Users\eazzi\.conda\envs\bw2\lib\site-packages\IPython\core\interactiveshell.py:3072: DtypeWarning: Columns (8,11,22) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
## Step 2) Create a dict for the database, with all the processes & products.
bw_project = 'agribalyse'
db_name = 'agribalyse'
data_process = [{
        'name': row['NAME'],
        'code': row['REF_ID'],
        'key': (db_name, row['REF_ID']),
        'database': db_name,
        'location': find_location_code(row, tbl_locations) , # parse something with row['F_LOCATION']
        'unit': '',
        'type': 'process',
        'exchanges':[],
        'description':row['DESCRIPTION'],
        'ID':row['ID'],
        'REF_ID':row['REF_ID'],
        'VERSION':row['VERSION'],
        'LAST_CHANGE':row['LAST_CHANGE'],
        'PROCESS_TYPE':row['PROCESS_TYPE'],
        'DEFAULT_ALLOCATION_METHOD':row['DEFAULT_ALLOCATION_METHOD'],
        'INFRASTRUCTURE_PROCESS':row['INFRASTRUCTURE_PROCESS'],
        'F_LOCATION':row['F_LOCATION'],
        } for i, row in tbl_processes.iterrows() ]
#  '':row[''],
db_data = {obj['key']: obj for obj in data_process}
#print(len(db_data))

## Create dicts for all the products, in the same database
data_products = [
    {'name': row['NAME'],
     'code': 'pro_'+row['REF_ID'],
     'key': (db_name, 'pro_'+row['REF_ID']),
     'type': 'product',
    }
    for i, row in tbl_flows.iterrows() if row['FLOW_TYPE'] == 'PRODUCT_FLOW'
]
db_data.update({obj['key']: obj for obj in data_products})
#print(len(db_data))

In [13]:
# How many processes? How many products?
print("There are {} processes.".format(len(data_process)))
print("There are {} products.".format(len(data_products)))
if len(data_products)>len(data_process):
    print("Warning: the matrix is not square!")
    print("{} more products than processes".format(len(data_products)-len(data_process)))
elif len(data_products)<len(data_process):
    print("Warning: the matrix is not square!")
    print("{} more processes than products".format(len(data_products)-len(data_process)))
else:
    print("Lucky you, the matrix is square!")

There are 15592 processes.
There are 16126 products.
534 more products than processes


In [22]:
## Step 3) Write it to a brightway project
#bw_project = 'agribalyse'
#db_name = 'agribalyse'
#bw2.projects
bw2.projects.set_current(bw_project)
agribalyse = bw2.Database(db_name)
agribalyse.write(db_data)

Writing activities to SQLite3 database:


Vacuuming database 


0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:05


Title: Writing activities to SQLite3 database:
  Started: 01/12/2022 14:51:48
  Finished: 01/12/2022 14:51:54
  Total time elapsed: 00:00:05
  CPU %: 76.70
  Memory %: 3.76


We can check that a few things work with that newly created database

In [23]:
bw_project = 'agribalyse'
bw2.projects.set_current(bw_project)

val = '00025e31-bdbd-360f-a0f0-574c3c34bca5' # melon, a process
bw2.Database(db_name).get(val).as_dict()

{'name': 'Melon, cantaloupe (ex Cavaillon or Charentais melon), pulp, raw, processed in FR | Ambient (average) | No packaging | No preparation | at consumer',
 'code': '00025e31-bdbd-360f-a0f0-574c3c34bca5',
 'key': ('agribalyse', '00025e31-bdbd-360f-a0f0-574c3c34bca5'),
 'database': 'agribalyse',
 'location': 'FR',
 'unit': '',
 'type': 'process',
 'description': 'Donnée AGRIBALYSE v3.0 ; 2020\nThis database has been produced as part of AGRIBALYSE program lead by ADEME and INRAE since 2009. It contains agricultural and food products produced and/or consumed in France. It follows CIQUAL nomenclature, the French nutritional database.  AGRIBALYSEv3.0 is built upon previous work and datasets, in particular AGRIBALYSE v1.3 and v1.4 (French agricultural products); ACYVIA (food transformation), Ecoinvent v3.5 (imported products + non-food-datasets) and WFLDB v3.1 (imported products). Methodology principles follow the key international guidelines as much as possible (ISO, LEAP, PEF). All deta

In [24]:
val = 'pro_3edbaa23-f0ef-31b1-8bc4-3ec9258a6f5b' #X-ray machine, a product (pro_)
bw2.Database(db_name).get(val).as_dict()

{'name': 'X-Ray machine, at plant/RER U (ACYVIA) - Dummy Process',
 'code': 'pro_3edbaa23-f0ef-31b1-8bc4-3ec9258a6f5b',
 'key': ('agribalyse', 'pro_3edbaa23-f0ef-31b1-8bc4-3ec9258a6f5b'),
 'type': 'product',
 'database': 'agribalyse'}

In [ ]:
## Step 4) Load a correspondance table for the biosphere
## Correspdance between the biosphere3 from bw2 & the biosphere from agribalyse
## To see how it was created, check code further down

## TO DO : the correspondance table is not complete yet 
# - some biosphere flows are not mapped, or it is not clear how to map them

In [21]:
biosphere_agri_loaded = pd.read_excel('correspondance_agribalyse_bio3_v2.xlsx',
                                      sheet_name='table',
                                     index_col=0)

In [39]:
## Step 5) Write exchanges into the database
# Long step - parsing about 800 000 exchanges
exc
fflow = exc['F_FLOW']
product = tbl_flows.query("ID == @fflow")
product['DESCRIPTION'].values[0]

'Imported from SimaPro\n\nWaste type: not defined'

In [ ]:
# write exchanges directly in bw2.database
biosphere3 = 'biosphere3'

c_procInvalid = 0 # counter for number of invalid processes
c_activityNotFound = 0 # counter for activities not found (should be 0 if all caught by c_procInvalid)
c_exchangeBioSkipped = 0
c_exchangeTechProdAdded = 0
c_exchangedSaved = 0

for i, exc in tbl_exchanges.iterrows():
    # loop on all exchanges... 
    
    # in which activity does this exchange take place
    fowner = exc['F_OWNER']
    process = tbl_processes.query("ID == @fowner")
    # check if there is an activity for real, otherwise skip that exchange
    if(len(process) == 0):
        c_procInvalid += 1
        #print('process not found in the process tables', fowner)
        # let's create a dummy process for it
        # assuming that there is a product describing it
        fflow = exc['F_FLOW']
        product = tbl_flows.query("ID == @fflow")
        newAct = agribalyse.new_activity(code=product['REF_ID'].values[0], # first the code, then some kwargs ad a dict passed to obj.update(kwargs)
                                name=product['NAME'].values[0],
                                type='process',
                                exchanges=[],
                                description=product['DESCRIPTION'].values[0],
                               )
        newAct.save()
        # continue 
        # it was the case for 3edbaa23-f0ef-31b1-8bc4-3ec9258a6f5b > X-Ray machine, at plant/RER U (ACYVIA) - Dummy Process 
        # a product exist, and exchanges exists, but no process exist for it      
        
    # get the activity, by code
    try:
        if(len(process) != 0):
            a = bw2.Database(db_name).get(process['REF_ID'].values[0]) # find activity
        else:
            fflow = exc['F_FLOW']
            product = tbl_flows.query("ID == @fflow")
            a = bw2.Database(db_name).get(product['REF_ID'].values[0]) # find activity
    except:
        c_activityNotFound += 1
        continue # go to next exchange/iteration
    
    amount = exc['RESULTING_AMOUNT_VALUE'] # the amount of the exchange, a float
    
    # which product-flow is it?
    fflow = exc['F_FLOW']
    product = tbl_flows.query("ID == @fflow")
    #print(type(product), len(product), product['FLOW_TYPE'])
    
    if(product['FLOW_TYPE'].values[0] == 'ELEMENTARY_FLOW'): # .values[0] needed because dtype was not set when reading the tbl_flows (cf warning above)
        # it is a biosphere exchange
        bio3 = biosphere_agri_loaded.query("ID == @fflow")
        exchange_type = 'biosphere'
        if( isinstance(bio3['bio3key'].values[0], str)):
            input_act = eval(bio3['bio3key'].values[0])
        else:
            # skip that exchange, go to next iteration
            c_exchangeBioSkipped += 1
            continue
        sign = 1
    else:
        # it is a technosphere exchange
        if(exc['IS_INPUT'] == 1): # an input
            exchange_type = 'technosphere'
            input_act = (db_name, 'pro_'+product['REF_ID'].values[0]) # refering to the 'product' entry in db_data
            sign = 1
            c_exchangeTechProdAdded += 1
        else: # an output
            exchange_type = 'production'
            input_act = (db_name, 'pro_'+product['REF_ID'].values[0]) # refering to the 'product' entry in db_data
            sign = 1
            c_exchangeTechProdAdded += 1
    exchange = {
        'amount':amount*sign,
        'type':exchange_type,
        'input':input_act,
    }
    
    exchange = a.new_exchange(amount = amount*sign, input = input_act, type = exchange_type)
    exchange.save()
    c_exchangedSaved += 1
    #print("exchange saved", exchange)
    
    
#    if(i>5):
#        break

In [42]:
c_exchangedSaved

4

In [43]:
c_activityNotFound

0

In [44]:
c_procInvalid

0

In [45]:
c_exchangeBioSkipped

2

In [46]:
c_exchangeTechProdAdded

2

In [33]:
len(tbl_flows[tbl_flows['FLOW_TYPE'] == 'PRODUCT_FLOW'])
# in activity browser: records in db 31 718 (ok), 
# but it tells me: 15 592 activities (col) and 17 241 products (rows)
# while I am supposed to have only 16 126: 1 115 products to many!

16126

## Writing db to bw2project

In [57]:
bw2.projects
bw2.projects.set_current('agribalyse')
agribalyse = bw2.Database(db_name)
agribalyse.write(db_data)

## Analysing created database

Why is it non-square with 1100 extra products?

In [47]:
agribalyse = bw2.Database(db_name)
len(agribalyse)

31718

In [44]:
# https://2.docs.brightway.dev/technical/bw2data.html?highlight=search#bw2data.backends.peewee.database.SQLiteBackend.search
agribalyse.search?

In [51]:
allproducts = [a for a in agribalyse
               if 'product' in a['type']]
allprocesses = [a for a in agribalyse
               if 'process' in a['type']]
anyother = [a for a in agribalyse
               if 'process' not in a['type']
               and 'product' not in a['type']]
len(allproducts), len(allprocesses), len(anyother) # correct length... 
# 16126 - 15592 = 534 : nb of products without corresponding activities...
# while, in ab-dev when doing an LCA: 17 241, which is 1649 products too much ... 


(16126, 15592, 0)

So what?

To get a square matrix: 
- filter out the products that are without corresponding activity, and save them in separate xlsx file (534 rows)
- make sure that there is a one to one correspondance between activities and products, after filter
- analyse which one were removed and why? is it possible to add a dummy activity producing them, instead?
> hopefully, after that operation, the activity-browser will not invent more rows out of nowhere : hypthesis: extra products in ab come from the exchange of type "production"

Exchanges to parse:
- without 0 exchanges
- should take about 7 hours with current method
- biosphere3 should be completed first, about 900 left to match or leave empty

Speed improvement of the parsing of exchanges:
- get rid of querries in the loop > pre-process with dictionaries and reversed dictionaries, if possible
- use multithread?


## Correspondance table between biospheres

In [9]:
# step 1 - get a clean table from the biosphere of agribalyse
# columns: id, name, compartment, cas 
data_biosphere = [
    {'name': row['NAME'],
     'code': 'pro_'+row['REF_ID'],
    }
    for i, row in tbl_flows.iterrows() if row['FLOW_TYPE'] == 'ELEMENTARY_FLOW'
]

In [10]:
biosphere_agri = tbl_flows.query("FLOW_TYPE == 'ELEMENTARY_FLOW'")
biosphere_agri = biosphere_agri.reset_index(drop=True)
biosphere_agri#['sub_compartment'] = []

,ID,REF_ID,NAME,VERSION,LAST_CHANGE,F_CATEGORY,SYNONYMS,DESCRIPTION,FLOW_TYPE,INFRASTRUCTURE_FLOW,CAS_NUMBER,FORMULA,F_REFERENCE_FLOW_PROPERTY,F_LOCATION
0,2974,172a8bf6-6556-11dd-ad8b-0800200c9a66,Silicon,1,1589190739412,2770,NaN,NaN,ELEMENTARY_FLOW,0,007440-21-3,NaN,740,NaN
1,2976,7ae371af-8532-11e0-9d78-0800200c9a66,"Coal, 26.4 MJ per kg",1,1589190739727,2770,NaN,Higher heating value (HHV): 27.8 MJ/kg\nLower ...,ELEMENTARY_FLOW,0,NaN,NaN,740,NaN
2,2978,99e5d02d-84e4-4b75-9e73-29bebc534c48,"Wood and wood waste, 9.5 MJ per kg",1,1589190740406,2774,NaN,NaN,ELEMENTARY_FLOW,0,NaN,NaN,740,NaN
3,2980,b77f96fb-ab33-42f5-b9de-a5c7b03eb62f,"Organic substances, unspecified",1,1589190692414,2775,NaN,NaN,ELEMENTARY_FLOW,0,NaN,NaN,740,NaN
4,2982,84b5009c-5b88-4cfe-b53f-6fbf3a0cba23,Kerosene,1,1588777141316,2766,NaN,NaN,ELEMENTARY_FLOW,0,064742-81-0,NaN,740,NaN
5,2984,7ae398bf-8532-11e0-9d78-0800200c9a66,"Oil, crude, 42 MJ per kg",1,1589190739522,2770,NaN,Higher heating value (HHV): 44.5 MJ/kg\nLower ...,ELEMENTARY_FLOW,0,NaN,NaN,740,NaN
6,2986,d02904aa-dc1a-43f2-86ce-b41aa9cab47c,Benzo(a)pyrene,1,1589190692119,2775,NaN,NaN,ELEMENTARY_FLOW,0,000050-32-8,NaN,740,NaN
7,2988,b09ea5f4-2f35-45a4-84fd-28f0e352919a,N-Nitrodimethylamine,1,1588777141435,2766,NaN,NaN,ELEMENTARY_FLOW,0,004164-28-7,NaN,740,NaN
8,2990,b85f6025-577e-3936-9569-14f021652fd7,"Uranium, 2291 GJ per kg",1,1589190739626,2770,NaN,Formula: U,ELEMENTARY_FLOW,0,007440-61-1,NaN,740,NaN
9,2992,b33b93b6-9d95-41f7-b9d3-5b97100bb7b9,Naphthalene,1,1589190691703,2775,NaN,NaN,ELEMENTARY_FLOW,0,000091-20-3,NaN,740,NaN


In [11]:
# what if there are more than two? 
# doesnt seem to ever happen - if there are several, they are in the same string with a '+'
list1 = []
list2 = []
list3 = []
for i, obj in biosphere_agri.iterrows():
    cat = obj['F_CATEGORY']
    row_category = tbl_categories.query("ID == @cat")
    cat_name = row_category['NAME'].values[0]
    hasMainCat = row_category['F_CATEGORY'].values[0]
    if hasMainCat:
        main_cat_name = tbl_categories.query("ID == @hasMainCat")['NAME'].values[0]
        
    else:
        main_cat_name = cat_name
    
    list1.append(cat_name)
    list2.append(main_cat_name)
    list3.append((main_cat_name, cat_name))

In [12]:
biosphere_agri['main_compartment'] = list2
biosphere_agri['sub_compartment'] = list1
biosphere_agri['full_compartment'] = list3

In [13]:
biosphere_agri.dropna(axis='columns', how='all', inplace=True)

In [14]:
biosphere_agri['main_compartment'].unique()

array(['Resource', 'Emission to water', 'Emission to air',
       'Emission to soil', 'Waste'], dtype=object)

In [15]:
biosphere_agri['sub_compartment'].unique()

array(['in ground', 'biotic', 'unspecified', 'ground water', 'river',
       'land', 'agricultural', 'in water', 'ocean',
       'high population density', 'lake', 'low population density',
       'in air', 'lower stratosphere + upper troposphere',
       'river, long-term', 'low population density, long-term',
       'industrial', 'ground water, long-term', 'forestry'], dtype=object)

In [16]:
biosphere_agri['bio3name'] = ['' for i in range(len(biosphere_agri))]
biosphere_agri['bio3key'] = ['' for i in range(len(biosphere_agri))]

In [17]:
biosphere_agri

,ID,REF_ID,NAME,VERSION,LAST_CHANGE,F_CATEGORY,DESCRIPTION,FLOW_TYPE,INFRASTRUCTURE_FLOW,CAS_NUMBER,F_REFERENCE_FLOW_PROPERTY,main_compartment,sub_compartment,full_compartment,bio3name,bio3key
0,2974,172a8bf6-6556-11dd-ad8b-0800200c9a66,Silicon,1,1589190739412,2770,NaN,ELEMENTARY_FLOW,0,007440-21-3,740,Resource,in ground,"(Resource, in ground)",,
1,2976,7ae371af-8532-11e0-9d78-0800200c9a66,"Coal, 26.4 MJ per kg",1,1589190739727,2770,Higher heating value (HHV): 27.8 MJ/kg\nLower ...,ELEMENTARY_FLOW,0,NaN,740,Resource,in ground,"(Resource, in ground)",,
2,2978,99e5d02d-84e4-4b75-9e73-29bebc534c48,"Wood and wood waste, 9.5 MJ per kg",1,1589190740406,2774,NaN,ELEMENTARY_FLOW,0,NaN,740,Resource,biotic,"(Resource, biotic)",,
3,2980,b77f96fb-ab33-42f5-b9de-a5c7b03eb62f,"Organic substances, unspecified",1,1589190692414,2775,NaN,ELEMENTARY_FLOW,0,NaN,740,Emission to water,unspecified,"(Emission to water, unspecified)",,
4,2982,84b5009c-5b88-4cfe-b53f-6fbf3a0cba23,Kerosene,1,1588777141316,2766,NaN,ELEMENTARY_FLOW,0,064742-81-0,740,Emission to air,unspecified,"(Emission to air, unspecified)",,
5,2984,7ae398bf-8532-11e0-9d78-0800200c9a66,"Oil, crude, 42 MJ per kg",1,1589190739522,2770,Higher heating value (HHV): 44.5 MJ/kg\nLower ...,ELEMENTARY_FLOW,0,NaN,740,Resource,in ground,"(Resource, in ground)",,
6,2986,d02904aa-dc1a-43f2-86ce-b41aa9cab47c,Benzo(a)pyrene,1,1589190692119,2775,NaN,ELEMENTARY_FLOW,0,000050-32-8,740,Emission to water,unspecified,"(Emission to water, unspecified)",,
7,2988,b09ea5f4-2f35-45a4-84fd-28f0e352919a,N-Nitrodimethylamine,1,1588777141435,2766,NaN,ELEMENTARY_FLOW,0,004164-28-7,740,Emission to air,unspecified,"(Emission to air, unspecified)",,
8,2990,b85f6025-577e-3936-9569-14f021652fd7,"Uranium, 2291 GJ per kg",1,1589190739626,2770,Formula: U,ELEMENTARY_FLOW,0,007440-61-1,740,Resource,in ground,"(Resource, in ground)",,
9,2992,b33b93b6-9d95-41f7-b9d3-5b97100bb7b9,Naphthalene,1,1589190691703,2775,NaN,ELEMENTARY_FLOW,0,000091-20-3,740,Emission to water,unspecified,"(Emission to water, unspecified)",,


In [ ]:
# step 2 - match with biosphere3 in bw2
# strategies: by cas-number x compartment; by name x compartment; by ...

In [18]:
def match_by_CAS_category(cas, main_cat, sub_cat):
    search_bio = [ a for a in bw2.Database('biosphere3')
                  if cas in a.as_dict()['CAS number']
                  and main_cat in a['categories']
                  and sub_cat in a['categories']
                  #and "cas" in a['CAS number'] # does not work if non type in CAS number
                  # unit conversion (!)
                 ]
    return search_bio    

In [19]:
category_map = {
    # agribalyse > biosphere3
    'Resource': 'natural resource',
    'Emission to water': 'water', # with 'type': 'emission'
    'Emission to air': 'air', # with 'type': 'emission'
    'Emission to soil': 'soil', # with 'type': 'emission'
    'Waste': '', # no correspondance?
    
    'in ground':'in ground',
    'biotic': 'biotic',
    'unspecified': '', # does not appear in compartment name explicitly
    'ground water': 'ground-',
    'river': 'surface water', #
    'land': 'land',
    'agricultural': 'agricultural',
    'in water': 'in water',
    'ocean': 'ocean',
    'high population density': '', # no equivalent
    'lake': 'surface water', #  
    'low population density': 'low population density',
    'in air':'in air',
    'lower stratosphere + upper troposphere':'lower stratosphere + upper troposphere',
    'river, long-term': '',
    'low population density, long-term': 'low population density long-term',
    'industrial':'industrial',
    'ground water, long-term': 'ground- long-term',
    'forestry':'forestry',
    '':'',
}

In [20]:
biosphere_agri.head()

,ID,REF_ID,NAME,VERSION,LAST_CHANGE,F_CATEGORY,DESCRIPTION,FLOW_TYPE,INFRASTRUCTURE_FLOW,CAS_NUMBER,F_REFERENCE_FLOW_PROPERTY,main_compartment,sub_compartment,full_compartment,bio3name,bio3key
0,2974,172a8bf6-6556-11dd-ad8b-0800200c9a66,Silicon,1,1589190739412,2770,NaN,ELEMENTARY_FLOW,0,007440-21-3,740,Resource,in ground,"(Resource, in ground)",,
1,2976,7ae371af-8532-11e0-9d78-0800200c9a66,"Coal, 26.4 MJ per kg",1,1589190739727,2770,Higher heating value (HHV): 27.8 MJ/kg\nLower ...,ELEMENTARY_FLOW,0,NaN,740,Resource,in ground,"(Resource, in ground)",,
2,2978,99e5d02d-84e4-4b75-9e73-29bebc534c48,"Wood and wood waste, 9.5 MJ per kg",1,1589190740406,2774,NaN,ELEMENTARY_FLOW,0,NaN,740,Resource,biotic,"(Resource, biotic)",,
3,2980,b77f96fb-ab33-42f5-b9de-a5c7b03eb62f,"Organic substances, unspecified",1,1589190692414,2775,NaN,ELEMENTARY_FLOW,0,NaN,740,Emission to water,unspecified,"(Emission to water, unspecified)",,
4,2982,84b5009c-5b88-4cfe-b53f-6fbf3a0cba23,Kerosene,1,1588777141316,2766,NaN,ELEMENTARY_FLOW,0,064742-81-0,740,Emission to air,unspecified,"(Emission to air, unspecified)",,


In [21]:
def match_by_name_category(name, main_cat, sub_cat, cas):
    
    search_bio = [ a for a in bw2.Database('biosphere3')
              if name in str(a)
              and category_map[main_cat] in str(a)#['categories']
              and category_map[sub_cat] in str(a)]#['categories']
              #and cas in a['CAS number']] # does not work if non type in CAS number
              # unit conversion - ? 
         
    return search_bio   

In [55]:
for i, row in biosphere_agri.iterrows():
    
    name = row['NAME']
    print(i, name)
    c1 = row['main_compartment']
    c2 = row['sub_compartment']
    cas = row['CAS_NUMBER']
    ooo = match_by_name_category(name, c1, c2, cas)
    print(cas)
    print(type(cas))
        
    print(isinstance(cas, float))
    print(isinstance(cas, str))
    
    print(ooo)
    
    if i > 10:
        break

0 Silicon
007440-21-3
<class 'str'>
False
True
['Silicon, in ground' (kilogram, None, ('natural resource', 'in ground'))]
1 Coal, 26.4 MJ per kg
nan
<class 'float'>
True
False
[]
2 Wood and wood waste, 9.5 MJ per kg
nan
<class 'float'>
True
False
[]
3 Organic substances, unspecified
nan
<class 'float'>
True
False
[]
4 Kerosene
064742-81-0
<class 'str'>
False
True
[]
5 Oil, crude, 42 MJ per kg
nan
<class 'float'>
True
False
[]
6 Benzo(a)pyrene
000050-32-8
<class 'str'>
False
True
['Benzo(a)pyrene' (kilogram, None, ('water',)), 'Benzo(a)pyrene' (kilogram, None, ('water', 'surface water'))]
7 N-Nitrodimethylamine
004164-28-7
<class 'str'>
False
True
[]
8 Uranium, 2291 GJ per kg
007440-61-1
<class 'str'>
False
True
[]
9 Naphthalene
000091-20-3
<class 'str'>
False
True
[]
10 Gas, natural, 46.8 MJ per kg
008006-14-2
<class 'str'>
False
True
[]
11 Water, KE
007732-18-5
<class 'str'>
False
True
[]


In [73]:
cs = 0
cns = 0
cm = 0
list_no_match = []
list_multi_match = []
for i, row in biosphere_agri.iterrows():
    
    name = row['NAME']
    print(i, name)
    c1 = row['main_compartment']
    c2 = row['sub_compartment']
    cas = row['CAS_NUMBER']
    ooo = match_by_name_category(name, c1, c2)
    
    # fix for 'unspecified' compartment, get a unique match
    if(c2 == 'unspecified' and len(ooo) > 1):
        ooa = [a for a in ooo if ',)' in str(a)]
        ooo = ooa
        
    if(len(ooo) == 1):
        cs+=1
        print("unique match")
        biosphere_agri.loc[i, 'bio3name'] = ooo[0]['name']
        biosphere_agri.loc[i, 'bio3key'] = str(ooo[0].key)
    elif(len(ooo) == 0):
        print("no match")
        cns+=1
        list_no_match.append((i, name))
    else: # more than one match
        print("more than one match")
        cm+=1
        list_multi_match.append((i, name+', '+c1+', '+c2, ooo))
        
    #if(i>100):
    #    break;
        
print("nb perfect match: ", cs)
print("nb multiple match: ", cm)
print("nb no match: ", cns)

0 Silicon
unique match
1 Coal, 26.4 MJ per kg
no match
2 Wood and wood waste, 9.5 MJ per kg
no match
3 Organic substances, unspecified
no match
4 Kerosene
no match
5 Oil, crude, 42 MJ per kg
no match
6 Benzo(a)pyrene
unique match
7 N-Nitrodimethylamine
no match
8 Uranium, 2291 GJ per kg
no match
9 Naphthalene
no match
10 Gas, natural, 46.8 MJ per kg
no match
11 Water, KE
no match
12 Water, KE
no match
13 Transformation, to annual crop, irrigated, extensive
unique match
14 Hydramethylnon
unique match
15 Ametryn
unique match
16 Bromacil
unique match
17 Fenamiphos
unique match
18 Water, BE
no match
19 Propachlor
unique match
20 Triforine
unique match
21 Hexaconazole
unique match
22 Water, HN
no match
23 Water, unspecified natural origin, HN
no match
24 Water, unspecified natural origin, CO
no match
25 Water, unspecified natural origin, BR
no match
26 Piperonyl butoxide
unique match
27 Transformation, from annual crop, greenhouse
unique match
28 Occupation, annual crop, greenhouse
unique m

unique match
251 Lactofen
unique match
252 Acifluorfen
unique match
253 Propiconazole
unique match
254 Acetochlor
unique match
255 Imazethapyr
unique match
256 Sulfosate
unique match
257 Carfentrazone-ethyl
unique match
258 Clethodim
unique match
259 Dimethenamid
unique match
260 Metolachlor
unique match
261 Chlorpyrifos
no match
262 Imidacloprid
unique match
263 Transformation, from shrub land, sclerophyllous
unique match
264 Nitrogen monoxide
no match
265 Methamidophos
unique match
266 Transformation, from forest, intensive
unique match
267 Thiophanate-methyl
no match
268 Benfluralin
unique match
269 Bromoxynil octanoate
unique match
270 Chlormephos
no match
271 Carboxin
unique match
272 Hexazinone
unique match
273 Occupation, agriculture
no match
274 Biological pest controls
no match
275 Organic chlorine compounds (unspecified)
no match
276 Peat
unique match
277 Rutile
no match
278 Halogenated Matter (unspecified)
no match
279 Aerosols, radioactive, unspecified
unique match
280 Cesi

no match
506 2-Chloroacetophenone
no match
507 VOC, volatile organic compounds as C
no match
508 Krypton-88
unique match
509 Antimony-124
unique match
510 Slags
no match
511 Thorium-234
unique match
512 Occupation, seabed, unspecified
no match
513 Carbon-14
unique match
514 Uranium-234
unique match
515 Nickel
more than one match
516 Gas, natural, 30.3 MJ per kg
no match
517 Radioactive species, alpha emitters
unique match
518 Cobalt-60
unique match
519 Thorium-232
unique match
520 Methane, trichlorofluoro-, CFC-11
unique match
521 Fluorine
no match
522 Ethene, chloro-
unique match
523 Antimony-125
unique match
524 Tellurium-123m
unique match
525 Copper
more than one match
526 Radium-224
unique match
527 Occupation, annual crop
no match
528 Production waste, not inert
no match
529 Cesium-136
unique match
530 Iodine-129
unique match
531 Hydrogen-3, Tritium
unique match
532 Sulfur trioxide
no match
533 Uranium ore, 1.11 GJ per kg
no match
534 Krypton-85
more than one match
535 Iodide
uniq

more than one match
748 Ammonia, FR
no match
749 Water
unique match
750 Waste water/m3
no match
751 Methane
more than one match
752 Water, MX
no match
753 Water, MX
no match
754 Water, IT
no match
755 Water, unspecified natural origin, IN
no match
756 Water, LK
no match
757 Captan
unique match
758 Water, LK
no match
759 Indoxacarb
unique match
760 Dichlorodimethylsilane
unique match
761 Silicon
unique match
762 Methane, monochloro-, R-40
unique match
763 Dimethyldichlorosilane
unique match
764 Dimethyldichlorosilane
unique match
765 Cyhalothrin
more than one match
766 Pronamide
unique match
767 Water, CH
no match
768 Water, unspecified natural origin, RER w/o CH+DE
no match
769 Water, unspecified natural origin, CH
no match
770 Buprofezin
unique match
771 Methiocarb
unique match
772 Metalaxyl-M
unique match
773 Kaolin
unique match
774 Mineral oil
unique match
775 Water, NZ
no match
776 Diazinon
unique match
777 Pyrimethanil
unique match
778 Water, NZ
no match
779 Dimethomorph
unique ma

unique match
982 Water, turbine use, unspecified natural origin, SE
no match
983 Cumene
unique match
984 Vanadium
unique match
985 Sulfur hexafluoride
more than one match
986 Ioxynil
unique match
987 Alachlor
no match
988 Acephate
unique match
989 Transformation, from annual crop, non-irrigated
more than one match
990 Rhodium, Rh 1.6E-7%, in mixed ore
unique match
991 Triethylene glycol
unique match
992 Carbon dioxide, non-fossil
unique match
993 Primisulfuron
unique match
994 Magnesium
unique match
995 Bromide
unique match
996 Water, IAI Area, Russia & RER w/o EU27 & EFTA
no match
997 2,4-D, dimethylamine salt
no match
998 Water, well, RNA
no match
999 Water, river, US
no match
1000 Water, PE
no match
1001 Water, RME
no match
1002 Particulates, < 2.5 um
unique match
1003 Chromium
more than one match
1004 Platinum
more than one match
1005 Lithium
more than one match
1006 Quizalofop ethyl ester
no match
1007 AOX, Adsorbable Organic Halogen as Cl
unique match
1008 Cinnabar
unique match
1

...

unique match
2083 Cloquintocet-mexyl
unique match
2084 Water, SI
no match
2085 Molybdenum, 0.014% in sulfide, Mo 8.2E-3% and Cu 0.81% in crude ore
unique match
2086 Butadiene
more than one match
2087 Water, river, PH
no match
2088 Chlorimuron-ethyl
no match
2089 Toluene
unique match
2090 Gangue, bauxite
unique match
2091 Nickel
no match
2092 Barium
more than one match
2093 Ethylene oxide
unique match
2094 Water, RU
no match
2095 Titanium
no match
2096 Barium
unique match
2097 Sulfate
no match
2098 Water, CN
no match
2099 Water, well, MY
no match
2100 Chloroacetyl chloride
unique match
2101 Lead
more than one match
2102 Transformation, to pasture, man made, intensive
unique match
2103 Water/m3
no match
2104 Hypochlorite
unique match
2105 Styrene
more than one match
2106 Propiconazole
unique match
2107 NMVOC, non-methane volatile organic compounds, unspecified origin
unique match
2108 Cadmium
no match
2109 Cadmium
unique match
2110 Sulfur dioxide
unique match
2111 Water, well, MX
no matc

unique match
2305 o-Xylene
unique match
2306 DOC, Dissolved Organic Carbon
unique match
2307 Terbufos
unique match
2308 Water, well, FR
no match
2309 Sulfate
no match
2310 Energy, gross calorific value, in biomass
more than one match
2311 Glyphosate
no match
2312 Water, IE
no match
2313 Titanium
unique match
2314 Water, GB
no match
2315 Potassium-40
more than one match
2316 PAH, polycyclic aromatic hydrocarbons
unique match
2317 Aluminium
no match
2318 Calcium
no match
2319 Lithium
unique match
2320 Isopropylamine
unique match
2321 Organic carbon
unique match
2322 Triallate
no match
2323 Fenoxaprop
no match
2324 Water, well, SE
no match
2325 Iodine
unique match
2326 Water, BE
no match
2327 Sulfur
unique match
2328 Fenpiclonil
unique match
2329 Water, unspecified natural origin, WEU
no match
2330 Methomyl
unique match
2331 Trifloxystrobin
no match
2332 Actinides, radioactive, unspecified
unique match
2333 Volume occupied, final repository for radioactive waste
unique match
2334 Barium
u

2539 Nickel
unique match
2540 Thallium
unique match
2541 Copper
no match
2542 Nitrogen, organic bound
unique match
2543 Oil, crude
unique match
2544 Polonium-210
more than one match
2545 Magnesium
more than one match
2546 Occupation, traffic area, road network
unique match
2547 Silver
more than one match
2548 Xenon-133
unique match
2549 Water, turbine use, unspecified natural origin, SK
no match
2550 Cobalt
no match
2551 Sodium
no match
2552 COD, Chemical Oxygen Demand
unique match
2553 Monocrotophos
unique match
2554 t-Butyl methyl ether
unique match
2555 Water, RU
no match
2556 Benzene
more than one match
2557 Titanium
no match
2558 Hydrogen
more than one match
2559 Magnesium
unique match
2560 Carbon monoxide, non-fossil
more than one match
2561 Uranium-235
unique match
2562 Carbon dioxide, fossil
more than one match
2563 Transformation, to seabed, infrastructure
unique match
2564 Zinc
no match
2565 Water, cooling, unspecified natural origin, HU
no match
2566 Diuron
unique match
2567

more than one match
2773 Tin
no match
2774 Water, lake, GLO
no match
2775 Fluorine
unique match
2776 Arsenic
unique match
2777 Water, CA
no match
2778 Cobalt
unique match
2779 Silver, 0.007% in sulfide, Ag 0.004%, Pb, Zn, Cd, In
unique match
2780 Perlite
unique match
2781 Water, well, GLO
no match
2782 Uranium-235
unique match
2783 Water, UCTE
no match
2784 Aluminium
unique match
2785 Water, unspecified natural origin, IAI Area, Russia & RER w/o EU27 & EFTA
no match
2786 Iodine-129
unique match
2787 Sulfuric acid
unique match
2788 Sulfosulfuron
unique match
2789 Methyl acrylate
more than one match
2790 Fluorine
unique match
2791 Water, AU
no match
2792 Ethane, 1,1,1,2-tetrafluoro-, HFC-134a
unique match
2793 Water, cooling, unspecified natural origin, ES
no match
2794 Water, RO
no match
2795 Zinc
no match
2796 Occupation, annual crop, non-irrigated, intensive
unique match
2797 Arsenic
no match
2798 Gold
more than one match
2799 Water, ZA
no match
2800 Benzo(k)fluoranthene
no match
2801

more than one match
3000 Urea
unique match
3001 Nickel
unique match
3002 Bromine
no match
3003 Silver-110
unique match
3004 Water, river, KR
no match
3005 Water, CO
no match
3006 Acenaphthene
unique match
3007 Thallium
more than one match
3008 Aldehydes, unspecified
more than one match
3009 Cesium-137
unique match
3010 Water, cooling, unspecified natural origin, CA
no match
3011 Furan
unique match
3012 Chlorine
unique match
3013 Gold, Au 4.3E-4%, in ore
unique match
3014 Water, DE
no match
3015 2-Nitrobenzoic acid
more than one match
3016 Granite
unique match
3017 Carbetamide
unique match
3018 Palladium, Pd 7.3E-4%, Pt 2.5E-4%, Rh 2.0E-5%, Ni 2.3E+0%, Cu 3.2E+0% in ore
no match
3019 Talc
unique match
3020 Water, CA
no match
3021 Ethene
unique match
3022 Antimony
more than one match
3023 Olivine
unique match
3024 Molybdenum
more than one match
3025 Hydrocarbons, chlorinated
unique match
3026 Monobutyltin
unique match
3027 Uranium-238
unique match
3028 Isoxaflutole
unique match
3029 Sili

- nb perfect match:  1581
- nb multiple match:  513
- nb no match:  1016

In [74]:
print("nb perfect match: ", cs)
print("nb multiple match: ", cm)
print("nb no match: ", cns)

nb perfect match:  1610
nb multiple match:  391
nb no match:  1109


In [207]:
list_no_match

[(1, 'Coal, 26.4 MJ per kg'),
 (2, 'Wood and wood waste, 9.5 MJ per kg'),
 (3, 'Organic substances, unspecified'),
 (4, 'Kerosene'),
 (5, 'Oil, crude, 42 MJ per kg'),
 (7, 'N-Nitrodimethylamine'),
 (8, 'Uranium, 2291 GJ per kg'),
 (9, 'Naphthalene'),
 (10, 'Gas, natural, 46.8 MJ per kg'),
 (11, 'Water, KE'),
 (12, 'Water, KE'),
 (18, 'Water, BE'),
 (22, 'Water, HN'),
 (23, 'Water, unspecified natural origin, HN'),
 (24, 'Water, unspecified natural origin, CO'),
 (25, 'Water, unspecified natural origin, BR'),
 (30, 'Correction flow for delayed emission of biogenic carbon dioxide'),
 (31, 'Copper oxide'),
 (32, 'Silicate particles'),
 (34, 'Water, well, FR'),
 (35, 'Water, cooling, unspecified natural origin, RoW'),
 (36, 'Sulfur'),
 (37, 'Water, cooling, unspecified natural origin, RER'),
 (38, 'Transformation, from forest, unspecified'),
 (39, 'Potashsalt, crude'),
 (40, 'Transformation, to mineral extraction site'),
 (41, 'Occupation, mineral extraction site'),
 (42, 'Fluoride'),
 (44

In [208]:
list_multi_match

[(76,
  'Haloxyfop, Emission to soil, agricultural',
  ['Haloxyfop-P-methyl' (kilogram, None, ('soil', 'agricultural')),
   'Haloxyfop- (R) Methylester' (kilogram, None, ('soil', 'agricultural'))]),
 (159,
  'Pyrethrin, Emission to soil, agricultural',
  ['Pyrethrin' (kilogram, None, ('soil', 'agricultural')),
   'Pyrethrine' (kilogram, None, ('soil', 'agricultural'))]),
 (166,
  'Occupation, permanent crop, non-irrigated, Resource, land',
  ['Occupation, permanent crop, non-irrigated, intensive' (square meter-year, None, ('natural resource', 'land')),
   'Occupation, permanent crop, non-irrigated' (square meter-year, None, ('natural resource', 'land')),
   'Occupation, permanent crop, non-irrigated, extensive' (square meter-year, None, ('natural resource', 'land'))]),
 (370,
  'Transformation, from grassland, Resource, land',
  ['Transformation, from grassland, natural (non-use)' (square meter, None, ('natural resource', 'land')),
   'Transformation, from grassland, natural, for lives

In [191]:
biosphere_agri

,ID,REF_ID,NAME,VERSION,LAST_CHANGE,F_CATEGORY,DESCRIPTION,FLOW_TYPE,INFRASTRUCTURE_FLOW,CAS_NUMBER,F_REFERENCE_FLOW_PROPERTY,main_compartment,sub_compartment,full_compartment,bio3name,bio3key
0,2974,172a8bf6-6556-11dd-ad8b-0800200c9a66,Silicon,1,1589190739412,2770,NaN,ELEMENTARY_FLOW,0,007440-21-3,740,Resource,in ground,"(Resource, in ground)","Silicon, in ground","('biosphere3', '00143719-33a7-5738-aa1b-131f97..."
1,2976,7ae371af-8532-11e0-9d78-0800200c9a66,"Coal, 26.4 MJ per kg",1,1589190739727,2770,Higher heating value (HHV): 27.8 MJ/kg\nLower ...,ELEMENTARY_FLOW,0,NaN,740,Resource,in ground,"(Resource, in ground)",,
2,2978,99e5d02d-84e4-4b75-9e73-29bebc534c48,"Wood and wood waste, 9.5 MJ per kg",1,1589190740406,2774,NaN,ELEMENTARY_FLOW,0,NaN,740,Resource,biotic,"(Resource, biotic)",,
3,2980,b77f96fb-ab33-42f5-b9de-a5c7b03eb62f,"Organic substances, unspecified",1,1589190692414,2775,NaN,ELEMENTARY_FLOW,0,NaN,740,Emission to water,unspecified,"(Emission to water, unspecified)",,
4,2982,84b5009c-5b88-4cfe-b53f-6fbf3a0cba23,Kerosene,1,1588777141316,2766,NaN,ELEMENTARY_FLOW,0,064742-81-0,740,Emission to air,unspecified,"(Emission to air, unspecified)",,
5,2984,7ae398bf-8532-11e0-9d78-0800200c9a66,"Oil, crude, 42 MJ per kg",1,1589190739522,2770,Higher heating value (HHV): 44.5 MJ/kg\nLower ...,ELEMENTARY_FLOW,0,NaN,740,Resource,in ground,"(Resource, in ground)",,
6,2986,d02904aa-dc1a-43f2-86ce-b41aa9cab47c,Benzo(a)pyrene,1,1589190692119,2775,NaN,ELEMENTARY_FLOW,0,000050-32-8,740,Emission to water,unspecified,"(Emission to water, unspecified)",,
7,2988,b09ea5f4-2f35-45a4-84fd-28f0e352919a,N-Nitrodimethylamine,1,1588777141435,2766,NaN,ELEMENTARY_FLOW,0,004164-28-7,740,Emission to air,unspecified,"(Emission to air, unspecified)",,
8,2990,b85f6025-577e-3936-9569-14f021652fd7,"Uranium, 2291 GJ per kg",1,1589190739626,2770,Formula: U,ELEMENTARY_FLOW,0,007440-61-1,740,Resource,in ground,"(Resource, in ground)",,
9,2992,b33b93b6-9d95-41f7-b9d3-5b97100bb7b9,Naphthalene,1,1589190691703,2775,NaN,ELEMENTARY_FLOW,0,000091-20-3,740,Emission to water,unspecified,"(Emission to water, unspecified)",,


In [124]:
search_bio = [ str(a) for a in bw2.Database('biosphere3')
              if "Silicon" in str(a)#['name'] 
              and "air" in str(a)
              #and "air" in a['categories'] 
              #and "cas" in a['CAS number'] # does not work if non type in CAS number
              # unit conversion
             ]
search_bio

["'Silicon' (kilogram, None, ('air', 'lower stratosphere + upper troposphere'))",
 "'Silicon tetrafluoride' (kilogram, None, ('air', 'urban air close to ground'))",
 "'Silicon tetrafluoride' (kilogram, None, ('air', 'low population density, long-term'))",
 "'Silicon tetrafluoride' (kilogram, None, ('air', 'lower stratosphere + upper troposphere'))",
 "'Silicon' (kilogram, None, ('air', 'non-urban air or from high stacks'))",
 "'Silicon tetrachloride' (kilogram, None, ('air', 'urban air close to ground'))",
 "'Silicon' (kilogram, None, ('air',))",
 "'Silicon tetrafluoride' (kilogram, None, ('air', 'non-urban air or from high stacks'))",
 "'Silicon tetrafluoride' (kilogram, None, ('air',))",
 "'Silicon' (kilogram, None, ('air', 'low population density, long-term'))",
 "'Silicon' (kilogram, None, ('air', 'urban air close to ground'))"]

In [110]:
search_bio[2].as_dict()['CAS number']

'130498-29-2'

In [42]:
# can we match them by emission values..? 
# as in - are there flows that are equivalent lca-calculation wise?
# ... how do i check that..

### CAS-version

In [22]:
def match_by_name_category(name, main_cat, sub_cat, cas):
    
    try: 
        search_bio = [ a for a in bw2.Database('biosphere3')
                  if name in str(a)
                  and category_map[main_cat] in str(a)#['categories']
                  and category_map[sub_cat] in str(a)#['categories']
                  and cas in a['CAS number']] # does not work if non type in CAS number
                  # unit conversion - ?
    except:
        search_bio = [ a for a in bw2.Database('biosphere3')
                  if name in str(a)
                  and category_map[main_cat] in str(a)#['categories']
                  and category_map[sub_cat] in str(a)]#['categories']
                  # unit conversion - ?   
        
    
    return search_bio    
    

In [73]:
cs = 0
cns = 0
cm = 0
list_no_match = []
list_multi_match = []
for i, row in biosphere_agri.iterrows():
    
    name = row['NAME']
    print(i, name)
    c1 = row['main_compartment']
    c2 = row['sub_compartment']
    cas = row['CAS_NUMBER']
    ooo = match_by_name_category(name, c1, c2, cas)
    
    # fix for 'unspecified' compartment, get a unique match
    if(c2 == 'unspecified' and len(ooo) > 1):
        ooa = [a for a in ooo if ',)' in str(a)]
        ooo = ooa
        
    if(len(ooo) == 1):
        cs+=1
        print("unique match")
        biosphere_agri.loc[i, 'bio3name'] = ooo[0]['name']
        biosphere_agri.loc[i, 'bio3key'] = str(ooo[0].key)
    elif(len(ooo) == 0):
        print("no match")
        cns+=1
        list_no_match.append((i, name))
    else: # more than one match
        print("more than one match")
        cm+=1
        list_multi_match.append((i, name+', '+c1+', '+c2, ooo))
        
    #if(i>100):
    #    break;
        
print("nb perfect match: ", cs)
print("nb multiple match: ", cm)
print("nb no match: ", cns)

0 Silicon
unique match
1 Coal, 26.4 MJ per kg
no match
2 Wood and wood waste, 9.5 MJ per kg
no match
3 Organic substances, unspecified
no match
4 Kerosene
no match
5 Oil, crude, 42 MJ per kg
no match
6 Benzo(a)pyrene
unique match
7 N-Nitrodimethylamine
no match
8 Uranium, 2291 GJ per kg
no match
9 Naphthalene
no match
10 Gas, natural, 46.8 MJ per kg
no match
11 Water, KE
no match
12 Water, KE
no match
13 Transformation, to annual crop, irrigated, extensive
unique match
14 Hydramethylnon
unique match
15 Ametryn
unique match
16 Bromacil
unique match
17 Fenamiphos
unique match
18 Water, BE
no match
19 Propachlor
unique match
20 Triforine
unique match
21 Hexaconazole
unique match
22 Water, HN
no match
23 Water, unspecified natural origin, HN
no match
24 Water, unspecified natural origin, CO
no match
25 Water, unspecified natural origin, BR
no match
26 Piperonyl butoxide
unique match
27 Transformation, from annual crop, greenhouse
unique match
28 Occupation, annual crop, greenhouse
unique m

unique match
251 Lactofen
unique match
252 Acifluorfen
unique match
253 Propiconazole
unique match
254 Acetochlor
unique match
255 Imazethapyr
unique match
256 Sulfosate
unique match
257 Carfentrazone-ethyl
unique match
258 Clethodim
unique match
259 Dimethenamid
unique match
260 Metolachlor
unique match
261 Chlorpyrifos
no match
262 Imidacloprid
unique match
263 Transformation, from shrub land, sclerophyllous
unique match
264 Nitrogen monoxide
no match
265 Methamidophos
unique match
266 Transformation, from forest, intensive
unique match
267 Thiophanate-methyl
no match
268 Benfluralin
unique match
269 Bromoxynil octanoate
unique match
270 Chlormephos
no match
271 Carboxin
unique match
272 Hexazinone
unique match
273 Occupation, agriculture
no match
274 Biological pest controls
no match
275 Organic chlorine compounds (unspecified)
no match
276 Peat
unique match
277 Rutile
no match
278 Halogenated Matter (unspecified)
no match
279 Aerosols, radioactive, unspecified
unique match
280 Cesi

no match
506 2-Chloroacetophenone
no match
507 VOC, volatile organic compounds as C
no match
508 Krypton-88
unique match
509 Antimony-124
unique match
510 Slags
no match
511 Thorium-234
unique match
512 Occupation, seabed, unspecified
no match
513 Carbon-14
unique match
514 Uranium-234
unique match
515 Nickel
more than one match
516 Gas, natural, 30.3 MJ per kg
no match
517 Radioactive species, alpha emitters
unique match
518 Cobalt-60
unique match
519 Thorium-232
unique match
520 Methane, trichlorofluoro-, CFC-11
unique match
521 Fluorine
no match
522 Ethene, chloro-
unique match
523 Antimony-125
unique match
524 Tellurium-123m
unique match
525 Copper
more than one match
526 Radium-224
unique match
527 Occupation, annual crop
no match
528 Production waste, not inert
no match
529 Cesium-136
unique match
530 Iodine-129
unique match
531 Hydrogen-3, Tritium
unique match
532 Sulfur trioxide
no match
533 Uranium ore, 1.11 GJ per kg
no match
534 Krypton-85
more than one match
535 Iodide
uniq

more than one match
748 Ammonia, FR
no match
749 Water
unique match
750 Waste water/m3
no match
751 Methane
more than one match
752 Water, MX
no match
753 Water, MX
no match
754 Water, IT
no match
755 Water, unspecified natural origin, IN
no match
756 Water, LK
no match
757 Captan
unique match
758 Water, LK
no match
759 Indoxacarb
unique match
760 Dichlorodimethylsilane
unique match
761 Silicon
unique match
762 Methane, monochloro-, R-40
unique match
763 Dimethyldichlorosilane
unique match
764 Dimethyldichlorosilane
unique match
765 Cyhalothrin
more than one match
766 Pronamide
unique match
767 Water, CH
no match
768 Water, unspecified natural origin, RER w/o CH+DE
no match
769 Water, unspecified natural origin, CH
no match
770 Buprofezin
unique match
771 Methiocarb
unique match
772 Metalaxyl-M
unique match
773 Kaolin
unique match
774 Mineral oil
unique match
775 Water, NZ
no match
776 Diazinon
unique match
777 Pyrimethanil
unique match
778 Water, NZ
no match
779 Dimethomorph
unique ma

unique match
982 Water, turbine use, unspecified natural origin, SE
no match
983 Cumene
unique match
984 Vanadium
unique match
985 Sulfur hexafluoride
more than one match
986 Ioxynil
unique match
987 Alachlor
no match
988 Acephate
unique match
989 Transformation, from annual crop, non-irrigated
more than one match
990 Rhodium, Rh 1.6E-7%, in mixed ore
unique match
991 Triethylene glycol
unique match
992 Carbon dioxide, non-fossil
unique match
993 Primisulfuron
unique match
994 Magnesium
unique match
995 Bromide
unique match
996 Water, IAI Area, Russia & RER w/o EU27 & EFTA
no match
997 2,4-D, dimethylamine salt
no match
998 Water, well, RNA
no match
999 Water, river, US
no match
1000 Water, PE
no match
1001 Water, RME
no match
1002 Particulates, < 2.5 um
unique match
1003 Chromium
more than one match
1004 Platinum
more than one match
1005 Lithium
more than one match
1006 Quizalofop ethyl ester
no match
1007 AOX, Adsorbable Organic Halogen as Cl
unique match
1008 Cinnabar
unique match
1

...

unique match
2083 Cloquintocet-mexyl
unique match
2084 Water, SI
no match
2085 Molybdenum, 0.014% in sulfide, Mo 8.2E-3% and Cu 0.81% in crude ore
unique match
2086 Butadiene
more than one match
2087 Water, river, PH
no match
2088 Chlorimuron-ethyl
no match
2089 Toluene
unique match
2090 Gangue, bauxite
unique match
2091 Nickel
no match
2092 Barium
more than one match
2093 Ethylene oxide
unique match
2094 Water, RU
no match
2095 Titanium
no match
2096 Barium
unique match
2097 Sulfate
no match
2098 Water, CN
no match
2099 Water, well, MY
no match
2100 Chloroacetyl chloride
unique match
2101 Lead
more than one match
2102 Transformation, to pasture, man made, intensive
unique match
2103 Water/m3
no match
2104 Hypochlorite
unique match
2105 Styrene
more than one match
2106 Propiconazole
unique match
2107 NMVOC, non-methane volatile organic compounds, unspecified origin
unique match
2108 Cadmium
no match
2109 Cadmium
unique match
2110 Sulfur dioxide
unique match
2111 Water, well, MX
no matc

unique match
2305 o-Xylene
unique match
2306 DOC, Dissolved Organic Carbon
unique match
2307 Terbufos
unique match
2308 Water, well, FR
no match
2309 Sulfate
no match
2310 Energy, gross calorific value, in biomass
more than one match
2311 Glyphosate
no match
2312 Water, IE
no match
2313 Titanium
unique match
2314 Water, GB
no match
2315 Potassium-40
more than one match
2316 PAH, polycyclic aromatic hydrocarbons
unique match
2317 Aluminium
no match
2318 Calcium
no match
2319 Lithium
unique match
2320 Isopropylamine
unique match
2321 Organic carbon
unique match
2322 Triallate
no match
2323 Fenoxaprop
no match
2324 Water, well, SE
no match
2325 Iodine
unique match
2326 Water, BE
no match
2327 Sulfur
unique match
2328 Fenpiclonil
unique match
2329 Water, unspecified natural origin, WEU
no match
2330 Methomyl
unique match
2331 Trifloxystrobin
no match
2332 Actinides, radioactive, unspecified
unique match
2333 Volume occupied, final repository for radioactive waste
unique match
2334 Barium
u

2539 Nickel
unique match
2540 Thallium
unique match
2541 Copper
no match
2542 Nitrogen, organic bound
unique match
2543 Oil, crude
unique match
2544 Polonium-210
more than one match
2545 Magnesium
more than one match
2546 Occupation, traffic area, road network
unique match
2547 Silver
more than one match
2548 Xenon-133
unique match
2549 Water, turbine use, unspecified natural origin, SK
no match
2550 Cobalt
no match
2551 Sodium
no match
2552 COD, Chemical Oxygen Demand
unique match
2553 Monocrotophos
unique match
2554 t-Butyl methyl ether
unique match
2555 Water, RU
no match
2556 Benzene
more than one match
2557 Titanium
no match
2558 Hydrogen
more than one match
2559 Magnesium
unique match
2560 Carbon monoxide, non-fossil
more than one match
2561 Uranium-235
unique match
2562 Carbon dioxide, fossil
more than one match
2563 Transformation, to seabed, infrastructure
unique match
2564 Zinc
no match
2565 Water, cooling, unspecified natural origin, HU
no match
2566 Diuron
unique match
2567

more than one match
2773 Tin
no match
2774 Water, lake, GLO
no match
2775 Fluorine
unique match
2776 Arsenic
unique match
2777 Water, CA
no match
2778 Cobalt
unique match
2779 Silver, 0.007% in sulfide, Ag 0.004%, Pb, Zn, Cd, In
unique match
2780 Perlite
unique match
2781 Water, well, GLO
no match
2782 Uranium-235
unique match
2783 Water, UCTE
no match
2784 Aluminium
unique match
2785 Water, unspecified natural origin, IAI Area, Russia & RER w/o EU27 & EFTA
no match
2786 Iodine-129
unique match
2787 Sulfuric acid
unique match
2788 Sulfosulfuron
unique match
2789 Methyl acrylate
more than one match
2790 Fluorine
unique match
2791 Water, AU
no match
2792 Ethane, 1,1,1,2-tetrafluoro-, HFC-134a
unique match
2793 Water, cooling, unspecified natural origin, ES
no match
2794 Water, RO
no match
2795 Zinc
no match
2796 Occupation, annual crop, non-irrigated, intensive
unique match
2797 Arsenic
no match
2798 Gold
more than one match
2799 Water, ZA
no match
2800 Benzo(k)fluoranthene
no match
2801

more than one match
3000 Urea
unique match
3001 Nickel
unique match
3002 Bromine
no match
3003 Silver-110
unique match
3004 Water, river, KR
no match
3005 Water, CO
no match
3006 Acenaphthene
unique match
3007 Thallium
more than one match
3008 Aldehydes, unspecified
more than one match
3009 Cesium-137
unique match
3010 Water, cooling, unspecified natural origin, CA
no match
3011 Furan
unique match
3012 Chlorine
unique match
3013 Gold, Au 4.3E-4%, in ore
unique match
3014 Water, DE
no match
3015 2-Nitrobenzoic acid
more than one match
3016 Granite
unique match
3017 Carbetamide
unique match
3018 Palladium, Pd 7.3E-4%, Pt 2.5E-4%, Rh 2.0E-5%, Ni 2.3E+0%, Cu 3.2E+0% in ore
no match
3019 Talc
unique match
3020 Water, CA
no match
3021 Ethene
unique match
3022 Antimony
more than one match
3023 Olivine
unique match
3024 Molybdenum
more than one match
3025 Hydrocarbons, chlorinated
unique match
3026 Monobutyltin
unique match
3027 Uranium-238
unique match
3028 Isoxaflutole
unique match
3029 Sili

In [74]:
print("nb perfect match: ", cs)
print("nb multiple match: ", cm)
print("nb no match: ", cns)

nb perfect match:  1610
nb multiple match:  391
nb no match:  1109


### just first word in name

In [75]:
name = 'Water, word word word'

In [78]:
name2 = ''
for i in name:
    if i == ',' or i == ' ':
        break
        
    else:
        name2 += i

In [79]:
name2

'Water'

In [23]:
def match_by_name_category(name, main_cat, sub_cat, cas):
    
    try: 
        search_bio = [ a for a in bw2.Database('biosphere3')
                  if name in str(a)
                  and category_map[main_cat] in str(a)#['categories']
                  and category_map[sub_cat] in str(a)#['categories']
                  and cas in a['CAS number']] # does not work if non type in CAS number
                  # unit conversion - ?
    except:
        search_bio = [ a for a in bw2.Database('biosphere3')
                  if name in str(a)
                  and category_map[main_cat] in str(a)#['categories']
                  and category_map[sub_cat] in str(a)]#['categories']
                  # unit conversion - ?   
    
    if search_bio == []: # if we still can't find anything
            name2 = ''
            for i in name:
                if i == ',' or i == ' ':
                    break
                else:
                    name2 += i
            
            if name2 != name: # check if they're the same
                
                    try: 
                        search_bio = [ a for a in bw2.Database('biosphere3')
                          if name2 in str(a)
                          and category_map[main_cat] in str(a)#['categories']
                          and category_map[sub_cat] in str(a)#['categories']
                          and cas in a['CAS number']] # does not work if non type in CAS number
                          # unit conversion - ?
                    except:
                        search_bio = [ a for a in bw2.Database('biosphere3')
                                  if name2 in str(a)
                                  and category_map[main_cat] in str(a)#['categories']
                                  and category_map[sub_cat] in str(a)]#['categories']
                                  # unit conversion - ? 
                
    
    return search_bio    
    

In [24]:
cs = 0
cns = 0
cm = 0
list_no_match = []
list_multi_match = []
for i, row in biosphere_agri.iterrows():
    
    name = row['NAME']
    if 'Water,' in name:
        # Water in biosphere3 does not have country code, to strip
        name = name[:len(name) - 5]
        
    print(i, name)
    c1 = row['main_compartment']
    c2 = row['sub_compartment']
    cas = row['CAS_NUMBER']
    ooo = match_by_name_category(name, c1, c2, cas)
    
    # fix for 'unspecified' compartment, get a unique match
    if(c2 == 'unspecified' and len(ooo) > 1):
        ooa = [a for a in ooo if ',)' in str(a)]
        ooo = ooa

        
    if(len(ooo) == 1):
        cs+=1
        print("unique match")
        biosphere_agri.loc[i, 'bio3name'] = ooo[0]['name']
        biosphere_agri.loc[i, 'bio3key'] = str(ooo[0].key)
    elif(len(ooo) == 0):
        print("no match")
        cns+=1
        list_no_match.append((i, name))
    else: # more than one match
        print("more than one match")
        cm+=1
        list_multi_match.append((i, name+', '+c1+', '+c2, ooo))
        
    #if(i>100):
    #    break;
        
print("nb perfect match: ", cs)
print("nb multiple match: ", cm)
print("nb no match: ", cns)

0 Silicon
unique match
1 Coal, 26.4 MJ per kg
more than one match
2 Wood and wood waste, 9.5 MJ per kg
more than one match
3 Organic substances, unspecified
more than one match
4 Kerosene
no match
5 Oil, crude, 42 MJ per kg
unique match
6 Benzo(a)pyrene
unique match
7 N-Nitrodimethylamine
no match
8 Uranium, 2291 GJ per kg
unique match
9 Naphthalene
no match
10 Gas, natural, 46.8 MJ per kg
more than one match
11 Water, KE
more than one match
12 Water, KE
unique match
13 Transformation, to annual crop, irrigated, extensive
unique match
14 Hydramethylnon
unique match
15 Ametryn
unique match
16 Bromacil
unique match
17 Fenamiphos
unique match
18 Water, BE
more than one match
19 Propachlor
unique match
20 Triforine
unique match
21 Hexaconazole
unique match
22 Water, HN
unique match
23 Water, unspecified natural origin, HN
more than one match
24 Water, unspecified natural origin, CO
more than one match
25 Water, unspecified natural origin, BR
more than one match
26 Piperonyl butoxide
unique

unique match
240 Trifloxystrobin
unique match
241 Flumioxazin
unique match
242 Flumetsulam
unique match
243 Pendimethalin
unique match
244 Rimsulfuron
unique match
245 Cyhalothrin
no match
246 Methoxyfenozide
unique match
247 Flufenacet
unique match
248 Fenoxaprop-P ethyl ester
unique match
249 Cypermethrin
unique match
250 Esfenvalerate
unique match
251 Lactofen
unique match
252 Acifluorfen
unique match
253 Propiconazole
unique match
254 Acetochlor
unique match
255 Imazethapyr
unique match
256 Sulfosate
unique match
257 Carfentrazone-ethyl
unique match
258 Clethodim
unique match
259 Dimethenamid
unique match
260 Metolachlor
unique match
261 Chlorpyrifos
no match
262 Imidacloprid
unique match
263 Transformation, from shrub land, sclerophyllous
unique match
264 Nitrogen monoxide
more than one match
265 Methamidophos
unique match
266 Transformation, from forest, intensive
unique match
267 Thiophanate-methyl
no match
268 Benfluralin
unique match
269 Bromoxynil octanoate
unique match
270 C

more than one match
487 Lanthanum-140
unique match
488 Ammonia
no match
489 Americium-241
unique match
490 Particulates, SPM
more than one match
491 Cesium-137
unique match
492 Uranium-235
unique match
493 Methane, tetrachloro-, CFC-10
more than one match
494 Xenon-131m
unique match
495 Technetium-99
unique match
496 Coal, 29.3 MJ per kg
more than one match
497 Radioactive species, other beta emitters
unique match
498 Ethane, dichloro-
no match
499 Hydrocarbons, aliphatic, alkenes, unspecified
more than one match
500 Waste, final, inert
no match
501 Sodium-24
unique match
502 Acenaphthylene
unique match
503 Palladium
more than one match
504 Methane, dichlorofluoro-, HCFC-21
unique match
505 Gas, natural, 35 MJ per m3
more than one match
506 2-Chloroacetophenone
no match
507 VOC, volatile organic compounds as C
unique match
508 Krypton-88
unique match
509 Antimony-124
unique match
510 Slags
no match
511 Thorium-234
unique match
512 Occupation, seabed, unspecified
no match
513 Carbon-14


no match
721 Water, unspecified natural origin/m3
more than one match
722 Metals, unspecified
no match
723 Benzene, chloro-
more than one match
724 Suspended solids, unspecified
no match
725 Water, salt, ocean
unique match
726 Nitrogen dioxide
no match
727 Furfural
unique match
728 Water, cooling, well
more than one match
729 Waste water
no match
730 Particulates
more than one match
731 Occupation, traffic area
more than one match
732 Water, ID
unique match
733 Formic acid
unique match
734 Water, cooling, unspecified natural origin/kg
more than one match
735 Furfural
unique match
736 Occupation, industrial area
unique match
737 Ethane, pentafluoro-, HFC-125
more than one match
738 Nitrogen, total
more than one match
739 Ethane, 1,1,1-trifluoro-, HFC-143a
more than one match
740 Phosphorus, total
unique match
741 Ethane, 1,1,2,2-tetrafluoro-, HFC-134
no match
742 Water, unspecified natural origin/m3
no match
743 Water
unique match
744 Ethane, 1,1,1-trifluoro-, HFC-143a
unique match
745 

unique match
938 Imazethapyr
unique match
939 Particulates, > 10 um
unique match
940 Acetone
unique match
941 Methane, chlorodifluoro-, HCFC-22
unique match
942 Copper, 1.42% in sulfide, Cu 0.81% and Mo 8.2E-3% in crude ore
unique match
943 Trinexapac-ethyl
unique match
944 Water, PT
unique match
945 Water, turbine use, unspecified natural origin, HU
more than one match
946 Uranium-238
more than one match
947 Propylamine
unique match
948 Water, PH
more than one match
949 Water, unspecified natural origin, IAI Area, Gulf Cooperation Council
more than one match
950 Beryllium
unique match
951 Water, well, IN
more than one match
952 Endosulfan
unique match
953 Dicamba
unique match
954 Hexane
more than one match
955 Chlorsulfuron
unique match
956 Thorium-232
unique match
957 Sulfur hexafluoride
unique match
958 Chloride
more than one match
959 Potassium-40
unique match
960 Lead-210
unique match
961 Oils, unspecified
unique match
962 Water, river, RER
more than one match
963 Tribenuron-methy

...

unique match
2211 Nickel
unique match
2212 Mancozeb
unique match
2213 Thorium-232
more than one match
2214 Water, CZ
unique match
2215 Water, unspecified natural origin, IAI Area, South America
more than one match
2216 Mecoprop-P
unique match
2217 Water, well, AU
more than one match
2218 Transformation, to forest, secondary (non-use)
unique match
2219 Hydrocarbons, aliphatic, unsaturated
unique match
2220 Prometryn
unique match
2221 Transformation, from grassland, natural, for livestock grazing
unique match
2222 Ethene, chloro-
unique match
2223 Propanal
unique match
2224 Transformation, from industrial area
unique match
2225 Mercury
unique match
2226 Carbon dioxide, from soil or biomass stock
unique match
2227 Water, UCTE without Germany
unique match
2228 Tellurium
more than one match
2229 Xylene
unique match
2230 Fluoranthene
no match
2231 t-Butylamine
more than one match
2232 Transformation, to annual crop, non-irrigated, intensive
unique match
2233 Tin
unique match
2234 Water, unsp

unique match
2420 Cumene
unique match
2421 Aluminium
unique match
2422 Nitrogen fluoride
more than one match
2423 Phosphorus
unique match
2424 Water, unspecified natural origin, OCE
more than one match
2425 Hydrocarbons, aromatic
unique match
2426 Tantalum
more than one match
2427 Fluorine
no match
2428 Cinidon-ethyl
unique match
2429 Cobalt-60
unique match
2430 o-Xylene
unique match
2431 Azoxystrobin
unique match
2432 Fenpropidin
unique match
2433 Lambda-cyhalothrin
unique match
2434 Pyraclostrobin (prop)
no match
2435 Dioxin, 2,3,7,8 Tetrachlorodibenzo-p-
unique match
2436 Ammonium, ion
more than one match
2437 Water, turbine use, unspecified natural origin, LU
more than one match
2438 Bifenox
unique match
2439 Ethane, 1,1,2-trichloro-1,2,2-trifluoro-, CFC-113
unique match
2440 Water, LT
unique match
2441 Benzene, ethyl-
unique match
2442 Benzene, 1,2-dichloro-
more than one match
2443 Methomyl
no match
2444 Water, AT
unique match
2445 Trifluralin
unique match
2446 Beryllium
more tha

unique match
2642 Boron
unique match
2643 Copper
unique match
2644 Propene
more than one match
2645 Aldehydes, unspecified
unique match
2646 Bensulfuron methyl ester
unique match
2647 Water, river, FR
more than one match
2648 Chlorosulfonic acid
unique match
2649 Phenol, pentachloro-
unique match
2650 Sodium chlorate
more than one match
2651 Butane
unique match
2652 Antimony-124
unique match
2653 Nitrite
unique match
2654 Dichlorprop
no match
2655 Water, ID
unique match
2656 Water, LV
unique match
2657 Barite
no match
2658 Chloride
unique match
2659 Polonium-210
unique match
2660 Tin
unique match
2661 Water, PG
unique match
2662 Carbonate
unique match
2663 Flumiclorac-pentyl
unique match
2664 2-Propanol
more than one match
2665 Benzo(g,h,i)perylene
no match
2666 Water, FI
more than one match
2667 Water, unspecified natural origin, PG
more than one match
2668 Butadiene
unique match
2669 Sulfate
more than one match
2670 Metazachlor
unique match
2671 Ethofumesate
unique match
2672 Sodium


more than one match
2866 Molybdenum
unique match
2867 Wood, unspecified, standing/m3
more than one match
2868 Selenium
unique match
2869 Water, IAI Area, EU27 & EFTA
more than one match
2870 Hypochlorite
unique match
2871 Titanium
no match
2872 Cobalt, Co 5.0E-2%, in mixed ore
unique match
2873 Hydrogen chloride
unique match
2874 Water, EE
unique match
2875 Copper
unique match
2876 Anhydrite
unique match
2877 Pyrene
no match
2878 Transformation, to annual crop, fallow
more than one match
2879 Acrolein
more than one match
2880 Propiconazole
unique match
2881 Radium-226
unique match
2882 Phosphate
unique match
2883 Water, NL
unique match
2884 Ammonium, ion
unique match
2885 Nickel
unique match
2886 Chromium
unique match
2887 Cesium-134
unique match
2888 Tebutam
unique match
2889 Water, IN
unique match
2890 Water, cooling, unspecified natural origin, IAI Area, Russia & RER w/o EU27 & EFTA
more than one match
2891 Mecoprop
unique match
2892 Tin
more than one match
2893 Molybdenum
more than

no match
3076 Propanal
unique match
3077 Methylamine
unique match
3078 Potassium
unique match
3079 Methanol
unique match
3080 Transformation, from pasture, man made
more than one match
3081 Manganese
unique match
3082 Occupation, grassland, natural (non-use)
unique match
3083 Ethane, 1,1,1,2-tetrafluoro-, HFC-134a
unique match
3084 Nitrogen, organic bound
unique match
3085 Transformation, from traffic area, rail/road embankment
unique match
3086 Water, turbine use, unspecified natural origin, US
more than one match
3087 Water, CN
unique match
3088 Nitrogen, atmospheric
unique match
3089 Occupation, arable land, unspecified use
unique match
3090 2-Methyl-4-chlorophenoxyacetic acid
no match
3091 Water, unspecified natural origin, RU
more than one match
3092 Thorium-230
unique match
3093 Nitrite
no match
3094 Gadolinium
more than one match
3095 Radium-228
unique match
3096 Transformation, to lake, artificial
unique match
3097 Butyric acid, 4-(2,4-dichlorophenoxy)-
no match
3098 Cadmium
un

In [25]:
print("nb perfect match: ", cs)
print("nb multiple match: ", cm)
print("nb no match: ", cns)

nb perfect match:  1802
nb multiple match:  810
nb no match:  498


In [26]:
biosphere_agri

,ID,REF_ID,NAME,VERSION,LAST_CHANGE,F_CATEGORY,DESCRIPTION,FLOW_TYPE,INFRASTRUCTURE_FLOW,CAS_NUMBER,F_REFERENCE_FLOW_PROPERTY,main_compartment,sub_compartment,full_compartment,bio3name,bio3key
0,2974,172a8bf6-6556-11dd-ad8b-0800200c9a66,Silicon,1,1589190739412,2770,NaN,ELEMENTARY_FLOW,0,007440-21-3,740,Resource,in ground,"(Resource, in ground)","Silicon, in ground","('biosphere3', '00143719-33a7-5738-aa1b-131f97..."
1,2976,7ae371af-8532-11e0-9d78-0800200c9a66,"Coal, 26.4 MJ per kg",1,1589190739727,2770,Higher heating value (HHV): 27.8 MJ/kg\nLower ...,ELEMENTARY_FLOW,0,NaN,740,Resource,in ground,"(Resource, in ground)",,
2,2978,99e5d02d-84e4-4b75-9e73-29bebc534c48,"Wood and wood waste, 9.5 MJ per kg",1,1589190740406,2774,NaN,ELEMENTARY_FLOW,0,NaN,740,Resource,biotic,"(Resource, biotic)",,
3,2980,b77f96fb-ab33-42f5-b9de-a5c7b03eb62f,"Organic substances, unspecified",1,1589190692414,2775,NaN,ELEMENTARY_FLOW,0,NaN,740,Emission to water,unspecified,"(Emission to water, unspecified)",,
4,2982,84b5009c-5b88-4cfe-b53f-6fbf3a0cba23,Kerosene,1,1588777141316,2766,NaN,ELEMENTARY_FLOW,0,064742-81-0,740,Emission to air,unspecified,"(Emission to air, unspecified)",,
5,2984,7ae398bf-8532-11e0-9d78-0800200c9a66,"Oil, crude, 42 MJ per kg",1,1589190739522,2770,Higher heating value (HHV): 44.5 MJ/kg\nLower ...,ELEMENTARY_FLOW,0,NaN,740,Resource,in ground,"(Resource, in ground)","Oil, crude, in ground","('biosphere3', '88d06db9-59a1-4719-9174-afeb1f..."
6,2986,d02904aa-dc1a-43f2-86ce-b41aa9cab47c,Benzo(a)pyrene,1,1589190692119,2775,NaN,ELEMENTARY_FLOW,0,000050-32-8,740,Emission to water,unspecified,"(Emission to water, unspecified)",Benzo(a)pyrene,"('biosphere3', 'd02904aa-dc1a-43f2-86ce-b41aa9..."
7,2988,b09ea5f4-2f35-45a4-84fd-28f0e352919a,N-Nitrodimethylamine,1,1588777141435,2766,NaN,ELEMENTARY_FLOW,0,004164-28-7,740,Emission to air,unspecified,"(Emission to air, unspecified)",,
8,2990,b85f6025-577e-3936-9569-14f021652fd7,"Uranium, 2291 GJ per kg",1,1589190739626,2770,Formula: U,ELEMENTARY_FLOW,0,007440-61-1,740,Resource,in ground,"(Resource, in ground)","Uranium, in ground","('biosphere3', '2ba5e39b-adb6-4767-a51d-90c1cf..."
9,2992,b33b93b6-9d95-41f7-b9d3-5b97100bb7b9,Naphthalene,1,1589190691703,2775,NaN,ELEMENTARY_FLOW,0,000091-20-3,740,Emission to water,unspecified,"(Emission to water, unspecified)",,


In [27]:
list_no_match

[(4, 'Kerosene'),
 (7, 'N-Nitrodimethylamine'),
 (9, 'Naphthalene'),
 (30, 'Correction flow for delayed emission of biogenic carbon dioxide'),
 (31, 'Copper oxide'),
 (32, 'Silicate particles'),
 (34, 'Water, well, FR'),
 (36, 'Sulfur'),
 (38, 'Transformation, from forest, unspecified'),
 (39, 'Potashsalt, crude'),
 (40, 'Transformation, to mineral extraction site'),
 (41, 'Occupation, mineral extraction site'),
 (42, 'Fluoride'),
 (46, 'Phosphate ore'),
 (47, 'Water, unspecified natural origin, ID'),
 (48, 'Water, unspecified natural origin, AU'),
 (49, 'Water, unspecified natural origin, IN'),
 (51, 'Water, unspecified natural origin, RNA'),
 (52, 'Water, unspecified natural origin, RME'),
 (53, 'Water, unspecified natural origin, RLA'),
 (54, 'Water, unspecified natural origin, RER'),
 (55, 'Water, unspecified natural origin, RAF'),
 (56, 'Water, unspecified natural origin, RU'),
 (57, 'Oxygen'),
 (58, 'Nitrogen, atmospheric'),
 (59, 'Water, unspecified natural origin, CN'),
 (60, '

In [28]:
list_multi_match

[(1,
  'Coal, 26.4 MJ per kg, Resource, in ground',
  ['Coal, hard, unspecified, in ground' (kilogram, None, ('natural resource', 'in ground')),
   'Coal, brown, in ground' (kilogram, None, ('natural resource', 'in ground'))]),
 (2,
  'Wood and wood waste, 9.5 MJ per kg, Resource, biotic',
  ['Wood, hard, standing' (cubic meter, None, ('natural resource', 'biotic')),
   'Wood, primary forest, standing' (cubic meter, None, ('natural resource', 'biotic')),
   'Wood, soft, standing' (cubic meter, None, ('natural resource', 'biotic')),
   'Wood, unspecified, standing' (cubic meter, None, ('natural resource', 'biotic'))]),
 (3,
  'Organic substances, unspecified, Emission to water, unspecified',
  ['DOC, Dissolved Organic Carbon' (kilogram, None, ('water',)),
   'AOX, Adsorbable Organic Halogen as Cl' (kilogram, None, ('water',)),
   'TOC, Total Organic Carbon' (kilogram, None, ('water',))]),
 (10,
  'Gas, natural, 46.8 MJ per kg, Resource, in ground',
  ['Gas, mine, off-gas, process, coal 

### save

In [32]:
biosphere_agri.to_excel('correspondance_agribalyse_bio3_v2_raw.xlsx', sheet_name='table')

In [30]:
import pickle

In [31]:
pickle.dump(list_no_match, open('list_no_match_v2.pickle', 'wb'))
pickle.dump(list_multi_match, open('list_multi_match_v2.pickle', 'wb'))